**Web Scraping**

**Create a program that extracts product information, such as names, prices, and ratings, from an online e-commerce website and stores the data in a structured format like a CSV file.**

**You are free to choose the website you scrape from.**

In [1]:
!pip install requests beautifulsoup4 pandas


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_page(url, headers, retries=3, backoff_factor=0.3):
    """Fetch a web page with retry logic."""
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response
        else:
            time.sleep(backoff_factor * (2 ** i))
    return None

# URL of the e-commerce category page (this is just an example)
url = 'https://www.amazon.com/s?k=laptops'

# Headers to mimic a browser visit (you may need to update these headers if the request gets blocked)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Fetch the web page
response = fetch_page(url, headers)

# Check if the request was successful
if response and response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all product containers (this will vary based on the website's structure)
    product_containers = soup.find_all('div', {'data-component-type': 's-search-result'})

    # Lists to store the extracted data
    product_names = []
    product_prices = []
    product_ratings = []

    # Extract the desired information from each product container
    for container in product_containers:
        # Extract the product name
        name = container.h2.a.text.strip()
        product_names.append(name)

        # Extract the product price
        price_whole = container.find('span', class_='a-price-whole')
        price_fraction = container.find('span', class_='a-price-fraction')
        if price_whole and price_fraction:
            price = price_whole.text + price_fraction.text
        else:
            price = 'N/A'
        product_prices.append(price)

        # Extract the product rating
        rating = container.find('span', class_='a-icon-alt')
        if rating:
            rating = rating.text.split(' ')[0]
        else:
            rating = 'N/A'
        product_ratings.append(rating)

    # Create a DataFrame using the extracted data
    data = {
        'Product Name': product_names,
        'Price': product_prices,
        'Rating': product_ratings
    }
    df = pd.DataFrame(data)

    # Save the data to a CSV file
    df.to_csv('products.csv', index=False)
    print('Data saved to products.csv')
else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code if response else "N/A"}')


Failed to retrieve the webpage. Status code: N/A
